# Workshop Week 10: Question Answering

#### Please follow the instructions in this code and the workshop Instructor.

Read the first part and try to understand the code. Then run it, look at the outputs, then complete the tasks.

Types of QA systems:

    Extractive QA systems: These systems extract the answer directly from the given text by identifying the relevant section of text that contains the answer.

    Abstractive QA systems: These systems generate a new answer by understanding the meaning of the question and synthesizing information from various sources.

Classical (before deep neural learning) QA systems:

    Information Retrieval based QA systems: These systems use information retrieval techniques to search for relevant documents and retrieve the most relevant answers.

    Knowledge Graph based QA systems: These systems represent information in a structured format and use graph-based algorithms to answer questions.

    Watson QA system: This system, developed by IBM, uses a combination of natural language processing, machine learning, and information retrieval techniques to answer questions in a wide range of domains.

Evaluation of QA and Stanford Question Answering Dataset (SQuAD):

SQuAD is a popular dataset used for evaluating QA systems. It consists of a large number of questions and answers, along with the corresponding passages of text that contain the answers. The dataset is used to evaluate the accuracy and performance of different QA systems.

Language models for QA systems:

    BiDAF (Bidirectional Attention Flow): This model uses a bidirectional attention mechanism to encode the question and the passage and identify the most relevant words and phrases.

    Encoder-decoder transformers: These models use transformer networks to encode the input text and generate the output answer.

    SpanBERT: This model is an extension of the BERT (Bidirectional Encoder Representations from Transformers) model and uses a span-based approach to answer questions. It considers all possible spans in the input text to generate the final answer.

In [1]:
# !pip install transformers
import torch
import torch.nn as nn
from transformers import AutoTokenizer, AutoModelForQuestionAnswering

# Load the BiDAF model and tokenizer
tokenizer = AutoTokenizer.from_pretrained('deepset/bert-base-cased-squad2')
model = AutoModelForQuestionAnswering.from_pretrained('deepset/bert-base-cased-squad2')

# Define a sample question and passage
question = "What is the capital of France?"
passage = "France, officially the French Republic, is a country primarily located in Western Europe, consisting of metropolitan France and several overseas regions and territories. Paris is the capital and most populous city of France."

# Encode the question and passage using the tokenizer
inputs = tokenizer.encode_plus(question, passage, return_tensors='pt', max_length=512, truncation=True, truncation_strategy='only_second')
input_ids = inputs['input_ids']
token_type_ids = inputs['token_type_ids']
attention_mask = inputs['attention_mask']

# Pass the encoded input through the BiDAF model
outputs = model(input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids, return_dict=True)
start_logits = outputs.start_logits
end_logits = outputs.end_logits

# Decode the predicted start and end positions to get the answer
start_index = torch.argmax(start_logits)
end_index = torch.argmax(end_logits) + 1

input_tokens = tokenizer.convert_ids_to_tokens(input_ids[0])
answer_tokens = input_ids[0][start_index:end_index]
answer = tokenizer.decode(answer_tokens)

# Skip over any tokens before the start position or after the end position
for i, token in enumerate(answer_tokens):
    if token == tokenizer.cls_token_id:
        start_index += 1
    elif token == tokenizer.sep_token_id:
        end_index -= 1
answer_tokens = input_ids[0][start_index:end_index]

# Decode the answer tokens to get the final answer
answer = tokenizer.decode(answer_tokens)
print("Answer:", answer)

C:\Users\samridhi\anaconda3\lib\site-packages\huggingface_hub\utils\_runtime.py:185: UserWarning: Pydantic is installed but cannot be imported. Please check your installation. `huggingface_hub` will default to not using Pydantic. Error message: '{e}'
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/152 [00:00<?, ?B/s]

C:\Users\samridhi\anaconda3\lib\site-packages\huggingface_hub\file_download.py:149: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\samridhi\.cache\huggingface\hub\models--deepset--bert-base-cased-squad2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config.json:   0%|          | 0.00/508 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/433M [00:00<?, ?B/s]

ChunkedEncodingError: ("Connection broken: ConnectionResetError(10054, 'An existing connection was forcibly closed by the remote host', None, 10054, None)", ConnectionResetError(10054, 'An existing connection was forcibly closed by the remote host', None, 10054, None))

## Task 1: Construct Q/A system

Task Description: In this task, you will be given a set of questions and a corresponding set of passages. Your goal is to use a QA model to find the answer to each question in its corresponding passage.

Please review the code to understand it, run the first part, and complete the rest to make a QA system.

Then follow the instructions from the workshop Instructor.


Example Questions and Passages:

Question 1: What is the capital of the United States?

Passage 1: The capital of the United States is Washington, D.C. It is located on the east coast of the country, and is home to many important government buildings and monuments.

Question 2: Who wrote the novel "To Kill a Mockingbird"?

Passage 2: "To Kill a Mockingbird" is a novel written by Harper Lee. It was published in 1960 and has since become a classic of American literature.

Question 3: What is the largest country in the world by area?

Passage 3: Russia is the largest country in the world by area. It covers more than 17 million square kilometers and spans 11 time zones.

Question 4: What is the capital of France?

Passage 4: Paris is the capital and most populous city of France. It is located in the north-central part of the country, and is known for its rich history, art, and culture.

Question 5: Who was the first president of the United States?

Passage 5: George Washington was the first president of the United States. He served from 1789 to 1797, and is widely regarded as one of the most important figures in American history.

In [ ]:
import torch
import torch.nn as nn
from transformers import AutoTokenizer, AutoModelForQuestionAnswering

# Load the QA model and tokenizer
model_name = "distilbert-base-cased-distilled-squad"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForQuestionAnswering.from_pretrained(model_name)

# Define a set of questions and passages
questions = [
    "What is the capital of the United States?",
    "Who wrote the novel \"To Kill a Mockingbird\"?",
    "What is the largest country in the world by area?",
    "What is the capital of France?",
    "Who was the first president of the United States?"
]
passages = [
    "The capital of the United States is Washington, D.C. It is located on the east coast of the country, and is home to many important government buildings and monuments.",
    "\"To Kill a Mockingbird\" is a novel written by Harper Lee. It was published in 1960 and has since become a classic of American literature.",
    "Russia is the largest country in the world by area. It covers more than 17 million square kilometers and spans 11 time zones.",
    "Paris is the capital and most populous city of France. It is located in the north-central part of the country, and is known for its rich history, art, and culture.",
    "George Washington was the first president of the United States. He served from 1789 to 1797, and is widely regarded as one of the most important figures in American history."
]

# Loop over each question and passage, and use the QA model to find the answer
for i, (question, passage) in enumerate(zip(questions, passages)):
    # Encode the question and passage using the tokenizer
    inputs = <your code>
    input_ids = inputs['input_ids']
    attention_mask = inputs['attention_mask']

    # Pass the encoded input through the QA model
    outputs = <your code>
    start_logits = <your code>
    end_logits = <your code>


    # Decode the predicted start and end positions to get the answer
    start_index = <your code>
    end_index = <your code>

    # Skip over any tokens before the start position or after the end position
    for j, token_id in enumerate(input_ids[0]):
        <your code>

    # Decode the answer from the corresponding tokens
    answer_tokens = <your code>
    answer = <your code>

    # Print the question, passage, and answer
    print("Question {}: {}".format(i+1, question))
    print("Passage {}: {}".format(i+1, passage))
    print("Answer {}: {}\n".format(i+1, answer))

## Task 2: Use the QA code above

Apply the code to one of Assignment 1 articles. Make a question, ground truth answer, and predict an answer using the code. Evaluate answer using precision/recall.